In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 导入数据集
train_df = pd.read_csv("/content/sample_data/train_values.csv")
train_labels_df = pd.read_csv("/content/sample_data/train_labels.csv")
test_df = pd.read_csv("/content/sample_data/test_values.csv")

# 将训练数据集和标签数据集进行合并
train_df = train_df.merge(train_labels_df, on="building_id")

# 将分类变量进行编码
train_df = pd.get_dummies(train_df, columns=['land_surface_condition', 'foundation_type', 'roof_type', 
                                             'ground_floor_type', 'other_floor_type', 'position', 
                                             'plan_configuration', 'legal_ownership_status'])

test_df = pd.get_dummies(test_df, columns=['land_surface_condition', 'foundation_type', 'roof_type', 
                                           'ground_floor_type', 'other_floor_type', 'position', 
                                           'plan_configuration', 'legal_ownership_status'])

# 将训练数据集分为训练集和验证集
X_train, X_valid, y_train, y_valid = train_test_split(train_df.drop(['damage_grade', 'building_id'], axis=1), 
                                                      train_df['damage_grade'], 
                                                      test_size=0.2, 
                                                      random_state=42)
train_df.select_dtypes(include=['object']).columns.tolist()


[]

In [ ]:

# 训练随机森林模型
rf_model = RandomForestClassifier(n_estimators=500, random_state=42)
rf_model.fit(X_train, y_train)

# 预测验证集的结果并计算RMSE值
valid_preds = rf_model.predict(X_valid)
rmse = mean_squared_error(y_valid, valid_preds, squared=False)

print("Validation RMSE: ", rmse)

# 使用训练好的模型来预测测试集
test_preds = rf_model.predict(test_df.drop(['building_id'], axis=1))

# 将预测结果保存到CSV文件中
submission_df = pd.DataFrame({'building_id': test_df['building_id'], 'damage_grade': test_preds})
submission_df.to_csv('submission.csv', index=False)



Validation RMSE:  0.5456994042824602
